In [1]:
!pip install -Uqq scikit-learn nltk

In [1]:
import json
from sklearn.model_selection import train_test_split
import numpy as np
import nltk
import random

#### Loading Data

In [42]:
domain_name = 'food'
with open(domain_name+'.jsonl', 'r') as file:
    lines = file.readlines()

raw_dataset = []

for i,line in enumerate(lines):
    try:
        data = json.loads(line.replace('\'', '\"'))
        raw_dataset.append(data)
        data['review'] = data['review'].replace('\"','')
        data['review'] = data['review'].replace("\'",'')
        data['excel_id'] = domain_name+'_'+str(int(i+1))
    except:
        print(i)

In [43]:
raw_dataset_size = len(raw_dataset)
print(f'dataset size is {raw_dataset_size}')

dataset size is 1917


#### extracting aspects

In [44]:
aspects_set = set()

In [45]:
for i, example in enumerate(raw_dataset):
    aspects = list(example['aspects'].keys())
    aspects_set.update(aspects)

In [46]:
aspects_set

{'ارزش خرید', 'ارزش غذایی', 'ارسال', 'بسته بندی', 'طعم', 'کیفیت'}

In [47]:
raw_dataset[51]

{'review': 'اصلا شیرین نیست یه کمی هم بوی سوختگی می ده',
 'sentiment': '-2',
 'category': 'حلوا شکری، ارده و کنجد',
 'aspects': {'طعم': '-2'},
 'excel_id': 'food_52'}

In [48]:
for entry in raw_dataset:
    if 'category' not in entry:
        print(entry['review_id'])
        raise Exception(entry['review'] + " does not have any category associated")

#### Defining Questions for aspects

In [49]:
aspects_candidate_words = {
    # For Foods"
    "ارزش خرید" : "قیمت و ارزش خرید",
    "بسته بندی" : "بسته بندی و نگهداری",
    "ارسال" : "ارسال و حمل و نقل",
    "کیفیت" : "کیفیت و تازگی",
    "ارزش غذایی" : "سلامت و ارزش غذایی",
    "طعم" : "عطر، بو، و طعم",
    # For Movies:
    "صدا" : "صداگذاری و جلوه های صوتی",
    "موسیقی" : "موسیقی",
    "داستان" : "داستان، فیلمنامه، دیالوگ ها و موضوع",
    "صحنه" : "گریم، طراحی صحنه و جلوه های ویژه ی بصری",
    "کارگردانی" : "تهیه، تدوین، کارگردانی و ساخت",
    "فیلمبرداری" : "فیلمبرداری و تصویربرداری",
    "بازی" : "شخصیت پردازی، بازیگردانی و بازی بازیگران"
}

In [50]:
aspects_questions = {}
for aspect in list(aspects_set):
    aspects_questions[aspect] = f'نظر شما در مورد {aspects_candidate_words[aspect]} این محصول چیست؟'

In [51]:
general_aspect_label = 'کلی'
aspects_questions[general_aspect_label] = 'نظر شما به صورت کلی در مورد این محصول چیست؟'

In [52]:
aspects_questions

{'ارزش غذایی': 'نظر شما در مورد سلامت و ارزش غذایی این محصول چیست؟',
 'طعم': 'نظر شما در مورد عطر، بو، و طعم این محصول چیست؟',
 'کیفیت': 'نظر شما در مورد کیفیت و تازگی این محصول چیست؟',
 'ارزش خرید': 'نظر شما در مورد قیمت و ارزش خرید این محصول چیست؟',
 'ارسال': 'نظر شما در مورد ارسال و حمل و نقل این محصول چیست؟',
 'بسته بندی': 'نظر شما در مورد بسته بندی و نگهداری این محصول چیست؟',
 'کلی': 'نظر شما به صورت کلی در مورد این محصول چیست؟'}

### Spliting Data

In [53]:
NONE_LABEL = -3

In [54]:
X = []
y = []
for rec in raw_dataset:
    X.append ({k:rec[k] for k in ('review','review_id','aspects','category','excel_id') if k in rec})
    y.append(rec['sentiment'])

In [55]:
X[0],y[0]

({'review': 'من یه مدته فقط از این محصول استفاده میکنم اما جدیدا دونه هاش درشت شده،و روی خود محصول هم نوشته تراریخته.',
  'aspects': {},
  'category': 'حبوبات و سویا',
  'excel_id': 'food_1'},
 '3')

In [56]:
test_dev_size_dic = {'movie':(0.2,0.11),'food':(0.1,0.11)}

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_dev_size_dic[domain_name][0], random_state=12, stratify=y)

In [58]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=test_dev_size_dic[domain_name][1], random_state=12, stratify=y_train)

In [59]:
len(X_train),len(X_test),len(X_valid)

(1535, 192, 190)

In [60]:
X_train[0],y_train[0]

({'review': 'دوستان حتما دقت کنید درقسمت فیله تکه های سبز رنگ داشت که نشان دهنده انتیبیوتیک هایی هست که به مرغ ها میدن و اون تکه سمیه. بیشترشم استخونه من راضی نبودم و دیگه ازین مارو خرید نمیکنم',
  'aspects': {'ارزش غذایی': '-2', 'کیفیت': '-2'},
  'category': 'گوشت مرغ',
  'excel_id': 'food_1744'},
 '-2')

### Setting Review_ID

In [61]:
review_id_counter=1
for x in X_train + X_valid + X_test:
    x['review_id'] = review_id_counter
    review_id_counter+=1

In [62]:
X_valid[1]

{'review': 'کلا که نوشابه محصول مفیدی نیست. اما اونایی که استفاده دارند توی شگفت قیمتش خوبه.',
 'aspects': {'ارزش غذایی': '-1', 'ارزش خرید': '1'},
 'category': 'نوشابه',
 'excel_id': 'food_5',
 'review_id': 1537}

### calculating similarities:

In [63]:
# bleu_avg = 0
# for entry_1 in X_train:
#     for entry_2 in X_test:
#         bleu_avg += nltk.translate.bleu_score.sentence_bleu(references=[entry_1['review']],hypothesis=entry_2['review'])
        
# bleu_avg = bleu_avg / (len(X_train)*len(X_test))
# print(f"BLEU = {bleu_avg}")

#### Putting X and y together:

In [64]:
raw_dataset_dic = dict()
for entry in zip(X_train,y_train):
    entry[0].update({'sentiment':entry[1]})
    
for entry in zip(X_test,y_test):
    entry[0].update({'sentiment':entry[1]})
    
for entry in zip(X_valid,y_valid):
    entry[0].update({'sentiment':entry[1]})

In [65]:
raw_dataset_dic = {
    'train': X_train,
    'test': X_test,
    'dev': X_valid
}

In [66]:
assert sum([len(dataset) for dataset in raw_dataset_dic.values()]) == raw_dataset_size

#### Creating new QA-ABSA dataset

In [67]:
def gen_question(aspect,aspects_questions,example,domain_name:str):
    
    if domain_name.lower()=='food':
        question = aspects_questions[aspect].replace('محصول',example['category'])
        
    elif domain_name.lower()=='movie':
        question = aspects_questions[aspect].replace('این محصول','فیلم ' + example['category'])

    else:
        raise Exception('Domain is not supported')
        
    return question

In [68]:
dataset_ABSA = {'train':list(),'dev':list(),'test':list()}
for dataset_name, dataset in raw_dataset_dic.items():

    for example in dataset:
        
        product_aspects = example['aspects'].keys()

        #aspect sentiments
        i=1
        for aspect in aspects_set:
            entry = {'review': example['review'],
                     'review_id': str(example['review_id']),
                     'example_id': str(i),
                     'excel_id':example['excel_id'],
                     'question': gen_question(aspect,aspects_questions,example,domain_name),
                     'category': example['category'],
                     'aspect': aspect,
                     'label': str(example['aspects'][aspect] if aspect in product_aspects else NONE_LABEL)
                    }
            
            dataset_ABSA[dataset_name].append(entry)
            i+=1

        # overal sentiment
        entry = {'review': example['review'],
                 'review_id': str(example['review_id']),
                 'example_id': str(i),
                 'excel_id':example['excel_id'],
                 'question': gen_question(general_aspect_label,aspects_questions,example,domain_name),
                 'category': example['category'],
                 'aspect': general_aspect_label,
                 'label': str(example['sentiment'])
                }
        
        dataset_ABSA[dataset_name].append(entry)

In [69]:
dataset_ABSA['train'][0:3]

[{'review': 'دوستان حتما دقت کنید درقسمت فیله تکه های سبز رنگ داشت که نشان دهنده انتیبیوتیک هایی هست که به مرغ ها میدن و اون تکه سمیه. بیشترشم استخونه من راضی نبودم و دیگه ازین مارو خرید نمیکنم',
  'review_id': '1',
  'example_id': '1',
  'excel_id': 'food_1744',
  'question': 'نظر شما در مورد سلامت و ارزش غذایی این گوشت مرغ چیست؟',
  'category': 'گوشت مرغ',
  'aspect': 'ارزش غذایی',
  'label': '-2'},
 {'review': 'دوستان حتما دقت کنید درقسمت فیله تکه های سبز رنگ داشت که نشان دهنده انتیبیوتیک هایی هست که به مرغ ها میدن و اون تکه سمیه. بیشترشم استخونه من راضی نبودم و دیگه ازین مارو خرید نمیکنم',
  'review_id': '1',
  'example_id': '2',
  'excel_id': 'food_1744',
  'question': 'نظر شما در مورد عطر، بو، و طعم این گوشت مرغ چیست؟',
  'category': 'گوشت مرغ',
  'aspect': 'طعم',
  'label': '-3'},
 {'review': 'دوستان حتما دقت کنید درقسمت فیله تکه های سبز رنگ داشت که نشان دهنده انتیبیوتیک هایی هست که به مرغ ها میدن و اون تکه سمیه. بیشترشم استخونه من راضی نبودم و دیگه ازین مارو خرید نمیکنم',
  're

#### Adding ID labeles

In [70]:
for dataset_name, dataset in dataset_ABSA.items():
    for i, example in enumerate(dataset):
        example['guid'] = f'{domain_name}-{dataset_name}-r{example["review_id"]}-e{example["example_id"]}'

#### Saving Data

In [71]:
for dataset_name, dataset in dataset_ABSA.items():
    
    with open(f'{domain_name}_{dataset_name}.jsonl', 'w') as f:
        
        for example in dataset:
            
#             f.write("%s\n" % annotation)
            f.write(f'{example}\n')
print('DONE')

DONE


### Merging the two datasets:

In [72]:
for dataset_split_name in ['train']:
    with open (f'ABSA_Dataset_{dataset_split_name}.jsonl','w') as out_file:
        for domain in ['food','movie']:
            with open (f'{domain}_{dataset_split_name}.jsonl') as in_file:
                for line in in_file.readlines():
                    out_file.write(line.strip()+'\n')